In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from Bio.PDB.PDBList import PDBList   # pip install biopython if import failure
import os
import os.path as osp
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from IPython import embed
import argparse
from Bio.PDB import PDBParser
import torch
from torch_geometric.data import Dataset, InMemoryDataset
from torch_geometric.data import HeteroData, Data
import rdkit.Chem as Chem    # conda install rdkit -c rdkit if import failure.
import logging
from torch_geometric.loader import DataLoader
from tqdm import tqdm    # pip install tqdm if fails.
import random

In [53]:
###csv顺序：test -> valid -> train
a1 = pd.read_csv('/home/jovyan/TankBind/examples/data/PRMT5/test.csv')
a2 = pd.read_csv('/home/jovyan/TankBind/examples/data/PRMT5/valid.csv')
a3 = pd.read_csv('/home/jovyan/TankBind/examples/data/PRMT5/train.csv')

In [54]:
s = pd.DataFrame()
s['smiles'] = pd.concat((a1['smiles'],a2['smiles'],a3['smiles']))
s['md5'] = s.index
s.to_csv('PRMT5-ID-SMILES.csv')

In [34]:
seed = 123
random.seed(seed)
# 没有使用GPU的时候设置的固定生成的随机数
np.random.seed(seed)
# 为CPU设置种子用于生成随机数，以使得结果是确定的
torch.manual_seed(seed)
# torch.cuda.manual_seed()为当前GPU设置随机种子
torch.cuda.manual_seed(seed)


In [36]:
print(torch.randn(1,5))
print(torch.randn(1,5))

tensor([[ 0.2103, -0.3908,  0.2350,  0.6653,  0.3528]])
tensor([[ 0.9728, -0.0386, -0.8861, -0.4709, -0.4269]])


In [2]:
cp_name = 'SOS1'
data_path = 'data'

In [38]:
with open(f'embedding/{cp_name}/{cp_name}_z_dic.pickle', 'rb') as f:
        z1_dic = pickle.load(f)

In [39]:
z1_dic

{'C[C@@H](NC(=O)c1cc(N2CCN(C(=O)C3CC3)CC2)nc2cc[nH]c12)c1cccc(C(F)F)c1F': tensor([[[ 2.3399, -1.4879,  4.8747,  ...,  2.3590,  1.1545,  8.3941],
          [ 1.8831, -1.2948,  4.9656,  ...,  2.3852,  1.1021,  9.0993],
          [ 1.7584, -1.0555,  4.5793,  ...,  2.3966,  1.3536,  9.0791],
          ...,
          [ 2.6375, -1.8197,  4.7907,  ...,  1.8962,  0.5390,  8.0581],
          [ 1.9908, -1.4390,  4.4469,  ...,  2.6538,  0.9489,  9.1351],
          [ 2.5197, -1.6800,  4.2525,  ...,  2.6062,  0.7709,  8.5632]],
 
         [[ 2.1851, -1.1090,  4.6696,  ...,  1.8702,  2.3108,  7.6009],
          [ 1.9387, -0.9416,  4.8662,  ...,  1.8577,  2.7454,  8.0793],
          [ 1.6759, -0.9517,  4.7765,  ...,  2.0617,  2.7283,  8.3647],
          ...,
          [ 3.0993, -1.9774,  4.6330,  ...,  1.9675,  2.4680,  6.7285],
          [ 2.5956, -1.4266,  4.3800,  ...,  1.8511,  2.3295,  8.0771],
          [ 2.9268, -1.5961,  4.1313,  ...,  1.8882,  2.2146,  7.9176]],
 
         [[ 2.3316, -1.7639

In [37]:
import pickle
with open('gasegasdg.pkl', 'rb') as f:
    z_yz_dic = pickle.load(f)
z_yz_dic[3]

tensor([[[ 2.2693, -1.4062,  4.0642,  ...,  2.3939,  1.4587,  7.8709],
         [ 2.0383, -1.2953,  3.9155,  ...,  2.2216,  1.7196,  8.8879],
         [ 1.9888, -1.2847,  3.9210,  ...,  2.2868,  1.7374,  8.9687],
         ...,
         [ 2.2467, -1.2667,  3.7199,  ...,  2.5209,  1.2137,  9.1198],
         [ 2.0149, -1.2938,  3.8197,  ...,  2.3803,  1.6675,  9.1490],
         [ 1.9888, -1.2847,  3.9210,  ...,  2.2868,  1.7374,  8.9687]],

        [[ 1.5851, -2.0898,  4.3281,  ...,  1.7419,  2.8498,  8.4402],
         [ 1.2736, -1.7670,  4.4192,  ...,  1.8751,  3.0511,  8.9658],
         [ 1.2783, -1.6859,  4.4080,  ...,  1.9093,  3.0217,  9.0674],
         ...,
         [ 2.2689, -1.4772,  4.0558,  ...,  1.8140,  2.4733,  8.7409],
         [ 1.4350, -1.4896,  4.3914,  ...,  1.9122,  2.8869,  9.1918],
         [ 1.2783, -1.6859,  4.4080,  ...,  1.9093,  3.0217,  9.0674]],

        [[ 2.5383, -1.9612,  4.8696,  ...,  0.8452,  1.6685,  8.1829],
         [ 2.0598, -1.7000,  5.1757,  ...,  1

In [35]:
import pickle
with open('gasegasdg-4.pkl', 'rb') as f:
    z_yzs_dic = pickle.load(f)

In [36]:
z_yzs_dic[3]

tensor([[[ 2.3399, -1.4879,  4.8747,  ...,  2.3590,  1.1545,  8.3941],
         [ 1.8831, -1.2948,  4.9656,  ...,  2.3852,  1.1021,  9.0993],
         [ 1.7584, -1.0555,  4.5793,  ...,  2.3966,  1.3536,  9.0791],
         ...,
         [ 2.6375, -1.8197,  4.7907,  ...,  1.8962,  0.5390,  8.0581],
         [ 1.9908, -1.4390,  4.4469,  ...,  2.6538,  0.9489,  9.1351],
         [ 2.5197, -1.6800,  4.2525,  ...,  2.6062,  0.7709,  8.5632]],

        [[ 2.1851, -1.1090,  4.6696,  ...,  1.8702,  2.3108,  7.6009],
         [ 1.9387, -0.9416,  4.8662,  ...,  1.8577,  2.7454,  8.0793],
         [ 1.6759, -0.9517,  4.7765,  ...,  2.0617,  2.7283,  8.3647],
         ...,
         [ 3.0993, -1.9774,  4.6330,  ...,  1.9675,  2.4680,  6.7285],
         [ 2.5956, -1.4266,  4.3800,  ...,  1.8511,  2.3295,  8.0771],
         [ 2.9268, -1.5961,  4.1313,  ...,  1.8882,  2.2146,  7.9176]],

        [[ 2.3316, -1.7639,  5.4850,  ...,  1.7563,  1.2861,  7.8664],
         [ 2.1933, -1.5838,  5.4901,  ...,  1

In [3]:
with open(f'embedding_1/{cp_name}/{cp_name}_z_dic.pickle', 'rb') as f:
        z1_dic = pickle.load(f)

In [ ]:
df_tr = pd.read_csv(f'{data_path}/{cp_name}/train.csv')
df_tr['smiles'][10]

In [11]:
z1_dic['C[C@@H](NC(=O)c1cc(N2CCN(C(=O)C3CC3)CC2)nc2cc[nH]c12)c1cccc(C(F)F)c1F'].shpae

NameError: name 'z1_dic' is not defined

In [19]:
z2_dic['Cc1nc(NC(C)c2cccc(C(F)(F)C(C)(C)O)c2F)c2cn(C3(C)CC3)c(=O)c(F)c2n1'][0]

tensor([[ 3.0132, -2.1145,  2.8687,  ...,  1.6953,  1.4559,  5.8407],
        [ 2.9870, -2.3818,  2.9469,  ...,  1.8012,  1.6740,  6.4748],
        [ 2.9428, -2.5652,  2.8479,  ...,  1.7798,  1.7830,  6.7796],
        ...,
        [ 2.7734, -2.3569,  2.9392,  ...,  1.4895,  1.5314,  6.3214],
        [ 3.2896, -2.5458,  3.5529,  ...,  2.1343,  2.1536,  7.2697],
        [ 3.0626, -2.4423,  3.2966,  ...,  1.8032,  1.7171,  6.8216]])

In [20]:
with open(f'embed/{cp_name}/{cp_name}_z_mask_dic.pickle', 'rb') as f:
        z_mask_dic = pickle.load(f)

In [3]:
with open(f'embedding/{cp_name}/{cp_name}_sum_embedding_dic.pickle', 'rb') as f1:
        SOS1_sum_embedding_dic = pickle.load(f1)
with open(f'embedding/{cp_name}/{cp_name}_z_dic.pickle', 'rb') as f:
        z_dic = pickle.load(f)
df_tr = pd.read_csv(f'{data_path}/{cp_name}/train.csv')
df_te = pd.read_csv(f'{data_path}/{cp_name}/test.csv')
df_va = pd.read_csv(f'{data_path}/{cp_name}/valid.csv')
df_tr['embed'] = df_te['embed'] = df_va['embed'] = ''
df_tr['z'] = df_te['z'] = df_va['z'] = ''
for index, row in df_tr.iterrows():
    row['embed'] = SOS1_sum_embedding_dic[row['smiles']]
    row['z'] = z_dic[row['smiles']]
    df_tr.iloc[index] = row
for index, row in df_te.iterrows():
    row['embed'] = SOS1_sum_embedding_dic[row['smiles']]
    row['z'] = z_dic[row['smiles']]
    df_te.iloc[index] = row
for index, row in df_va.iterrows():
    row['embed'] = SOS1_sum_embedding_dic[row['smiles']]
    row['z'] = z_dic[row['smiles']]
    df_va.iloc[index] = row

In [33]:
a = df_tr.loc[:, 'z'].to_numpy()[0]
b = df_tr.loc[:, 'z'].to_numpy()[1]

In [37]:
a1 = torch.unsqueeze(a, dim=0)
b1 = torch.unsqueeze(b, dim=0)

In [38]:
torch.concat((a1,b1))

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 34 but got size 35 for tensor number 1 in the list.

In [16]:
torch.from_numpy(np.array(embed)).to(torch.double)

tensor([  7295.5718,  -6179.6108,   7985.6260,  10558.9609,  -3863.0754,
          1874.1375,    612.0879,  -5922.7700,  -1991.2085,   3476.6892,
          7682.9136,   5522.0137,   6655.0601,   9226.5605,  -2289.0125,
          7817.2446,   3434.8013,    216.1904,   5004.5522,   7140.1074,
         -9133.6318,   9270.3848,  16789.3164,   3299.7046, -20606.5664,
         -7122.0000,  38028.9727,   2258.1685,  -4636.8979,   7467.6230,
          1952.0891,  12160.7871,  12557.6816,  -8187.3975,  -3658.1338,
         14071.5908,   9626.4844,   4395.8291,  -1401.1707,   4924.4038,
          -188.0455,   5632.4868,   4946.7588,    440.8533,   5278.3477,
          -161.5130,  19907.0391,   3824.0552,  11751.7500,   4895.6064,
         -3015.1365,   2540.9653,   8079.2974,   5214.3828, -19366.1289,
         -5080.1797,   5507.0752,   1829.8391,   6824.9844,  -8244.6738,
           687.9783,  -4938.0981,   1040.7375,  -9033.2793,  -5148.1650,
           393.4572,  -1199.0422,   7579.3418,  -63

In [55]:
class TBDataset(Dataset):
    def __init__(self,root, df_tr=None, df_te=None, df_va=None,
                 transform=None, pre_transform=None, pre_filter=None):
        '''
        '''
        self.compound_name = 'SOS1'
        self.trainindx = 0
        self.testindx = 0
        self.valindx = 0
        self.test_df = df_te
        self.train_df = df_tr
        self.val_df = df_va
        self.testindx = len(self.test_df)
        self.expected_test_count = len(self.test_df)
        self.trainindx = len(self.train_df) +  self.testindx
        self.expected_train_count = len(self.train_df)
        self.valindx = len(self.val_df) + self.trainindx
        self.expected_val_count = len(self.val_df) 
        super(TBDataset, self).__init__(root, transform, pre_transform, pre_filter)
        print(self.processed_dir)


    @property
    def processed_file_names(self):
        return ['data_0.pt']
    def process(self):
        tasks = 'label'
        idx_list = []
        i = 0
        for df in [self.test_df, self.train_df, self.val_df]:
            # print(df)
            smiles_list = df.loc[:, 'smiles'].tolist() if 'smiles' in set(df.columns) else df.loc[:, 'mol'].tolist() if 'mol' in set(df.columns) else None
            assert smiles_list is not None, "Cannot find the smiles column in the data file"
            assert tasks is not None, "Cannot determine the target tasks due to unsupported dataset"
            self.ntasks = len(tasks)
            labels_list = df.loc[:, tasks]
            group_id_list = df.loc[:, 'group']
            embed_list = df.loc[:, 'embed']
            z_list = df.loc[:, 'z']
            
            labels_list = labels_list.to_numpy()
            group_id_list = group_id_list.to_numpy()
            embed_list = embed_list.to_numpy()
            z_list = z_list.to_numpy()
            group_dic = {}
            tmp = 0
            for idx in tqdm(range(len(smiles_list))):
                embed, label, group_id_str, z = embed_list[idx], labels_list[idx], group_id_list[idx], z_list[idx]
                # embed, label, group_id_str = embed_list[idx], labels_list[idx], group_id_list[idx]
                data = HeteroData()
                data.x = embed
                data.x = data.x.reshape(1,128)
                data.z = z.view(-1,128)
                data.y = torch.from_numpy(np.array(label)).to(torch.double)
                data.reverse = 0
                data.reverse = torch.tensor([data.reverse],dtype = torch.int64) 
                group_id = group_dic.get(group_id_str)
                if group_id is None:
                    group_id = tmp
                    group_dic[group_id_str] = tmp
                    tmp += 1
                data.group_id =  torch.from_numpy(np.array(group_id)).to(torch.double)

                torch.save(data, osp.join(self.processed_dir, 'data_{}.pt'.format(i)))
                i += 1
            idx_list.append(i)

            
        self.testindx, self.trainindx, self.valindx = idx_list
        self.expected_test_count = self.testindx
        self.trainindx = len(self.train_df) +  self.testindx
        self.expected_train_count = len(self.train_df)
        self.valindx = len(self.val_df) + self.trainindx
        self.expected_val_count = len(self.val_df)
        
    def len(self):
        return self.expected_test_count + self.expected_train_count \
            + self.expected_val_count
    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'data_{}.pt'.format(idx)))
        data.idx = idx
        return  data
    def get_idx_split(self):
        return self.expected_test_count, \
            self.expected_test_count + self.expected_train_count, \
            self.expected_test_count + self.expected_train_count + self.expected_val_count

In [5]:
class TBNet(nn.Module):
    def __init__(self):
        super(TBNet, self).__init__()
        self.downstream_n_layers = 3
        self.lr = 0.0001
        self.dropout = 0.15
        self.compound_name = 'SOS1'
        self.loss_fn = PairwiseLoss()
        self.downstream_out = nn.Linear(128, 1)
        ffn_layers=[FFN(128, 0.15) for _ in range(3)]
        self.layers = nn.ModuleList(ffn_layers)

        
    def forward(self, x):
        for ffn_layer in self.layers:
            x = ffn_layer(x)
        x = self.downstream_out(x)
        return x

    def my_loss(self, pred, target, groupid):
        is_valid = torch.logical_not(torch.isnan(target))
        return self.loss_fn(pred[is_valid], target[is_valid], groupid)
class PairwiseLoss(nn.Module):
    def __init__(self, keep_rate=1., sigmoid_lambda=1.,
                 ingrp_thr=0.3, outgrp_thr=9999, eval=False):
        super(PairwiseLoss, self).__init__()
        self.eval = eval
        self.register_buffer('keep_rate', torch.tensor(keep_rate, dtype=torch.float64))
        self.register_buffer('sigmoid_lambda', torch.tensor(sigmoid_lambda, dtype=torch.float64))
        self.register_buffer('ingrp_thr', torch.tensor(ingrp_thr, dtype=torch.float64))
        self.register_buffer('outgrp_thr', torch.tensor(outgrp_thr, dtype=torch.float64))

    def forward(self, pred, true, groupid):

        """
        Customized pairwise ranking loss.

        """
        # print('pred',pred)
        # print('true',true)
        # print('group_id',groupid)
        if len(pred.shape) == 1:
            pred = pred.unsqueeze(1)
        if len(true.shape) == 1:
            true = true.unsqueeze(1)
        drop_rate = 1 - self.keep_rate
        true_tile_row = true.repeat((1, true.shape[0]))
        true_tile_col = torch.t(true_tile_row)
        assert (true_tile_row.shape == true_tile_col.shape)

        groupid_row = groupid.repeat((groupid.shape[0], 1))
        # print(groupid_row.shape)
        groupid_col = torch.t(groupid_row)

        pred_tile_row = pred.repeat((1, pred.shape[0]))
        pred_tile_col = torch.t(pred_tile_row)
        # diff = (true_tile_row - true_tile_col) / (torch.abs(true_tile_col) + 1e-4)
        diff = (true_tile_row - true_tile_col)
        # print(diff)
        pred_pair = torch.stack([pred_tile_row, pred_tile_col], dim=0)
        # print(groupid_row.shape)
        # print(groupid_col.shape)
        # print(diff.shape)
        valid_ind = torch.logical_or(torch.logical_and(groupid_row == groupid_col, diff > self.ingrp_thr),
                                     diff > self.outgrp_thr)
        # print("(valid_ind)",valid_ind)
        pred_pair_valid = pred_pair.masked_select(valid_ind).reshape(2, -1)
        pred_pair_diff = pred_pair_valid[0] - pred_pair_valid[1]
        # print(pred_pair_diff)
        reverse = torch.sum(pred_pair_diff > 0)
        ntotal = pred_pair_valid.shape[1] + 1e-8
        # print("Total valid pairs: {:.3f}, reversed: {:.3f}, reverse ratio: {:.3f}".format(ntotal, reverse, reverse/ntotal))
        if drop_rate > 1e-4:
            pred_pair_valid_dropout = torch.nn.functional.dropout(pred_pair_valid[0], drop_rate) * self.keep_rate
            pred_pair_valid_ind = torch.logical_or(pred_pair_valid_dropout == pred_pair_valid[0],
                                                   pred_pair_valid_dropout != 0.0)

            pred_pair_valid = pred_pair_valid.masked_select(pred_pair_valid_ind).reshape(2, -1)

        loss = torch.sum(torch.log(1. + torch.exp(self.sigmoid_lambda * (pred_pair_valid[1] - pred_pair_valid[0]))))
        
        num = pred_pair_valid.shape[1] + 1e-8

        loss = torch.div(loss, num)
        return loss, reverse/ntotal
class FFN(nn.Module):
    def __init__(self, hidden_size, dropout_rate):
        super(FFN, self).__init__()

        self.layer1 = nn.Linear(hidden_size, hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(p=dropout_rate)


    def forward(self, x):
        y = self.layer1(x)
        y = self.gelu(y)
        y = self.dropout(y)
        return y

In [9]:
df_tr


KeyboardInterrupt



In [56]:
dataset = TBDataset(root='data/SOS1',df_tr=df_tr, df_te=df_te, df_va=df_va)

Processing...
100%|██████████| 191/191 [00:07<00:00, 26.11it/s]

data/SOS1/processed



Done!


In [57]:
dataset[0]

HeteroData(
  x=[1, 128],
  z=[5075, 128],
  y=7.4591703,
  reverse=[1],
  group_id=0.0,
  idx=0
)

In [58]:
dataset[1]

HeteroData(
  x=[1, 128],
  z=[4495, 128],
  y=7.4936304,
  reverse=[1],
  group_id=0.0,
  idx=1
)

In [59]:
te, tr, va = dataset.get_idx_split()
dataset_test = dataset[:te]
dataset_train = dataset[te:tr]
#self.dataset_train= self.dataset_test
#self.dataset_val =self.dataset_test
dataset_val = dataset[tr:va]

In [60]:
data_loader_tr = DataLoader(dataset_train, batch_size=3, shuffle=False, num_workers=3)
data_loader_te = DataLoader(dataset_test, batch_size=10, shuffle=False, num_workers=3)
data_loader_va = DataLoader(dataset_val, batch_size=10, shuffle=False, num_workers=3)
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = TBNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [61]:
dataset_train[0]

HeteroData(
  x=[1, 128],
  z=[4930, 128],
  y=8.69897,
  reverse=[1],
  group_id=0.0,
  idx=119
)

In [62]:
dataset_train[1]

HeteroData(
  x=[1, 128],
  z=[5075, 128],
  y=8.69897,
  reverse=[1],
  group_id=0.0,
  idx=120
)

In [24]:
x = dataset_train[0].x.to(device)
x1 = dataset_train[1].x.to(device)
x = torch.concat((x,x1))
model(x)

tensor([[  85.3664],
        [-194.5437]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [65]:
for data in tqdm(data_loader_tr):
    print(data)
    y = data.y.to(device)
    x = data.x.to(device)
    data.group_id = data.group_id.to(device)
    outputs = model(x)
    print(outputs.shape)
    break

  0%|          | 0/190 [00:00<?, ?it/s]

HeteroDataBatch(
  x=[3, 128],
  z=[14210, 128],
  y=[3],
  reverse=[3],
  group_id=[3],
  idx=[3]
)
torch.Size([3, 1])


  0%|          | 0/190 [00:01<?, ?it/s]


In [28]:
for epoch in range(30):
    for data in tqdm(data_loader_tr):
        y = data.y.to(device)
        x = data.x.to(device)
        data.group_id = data.group_id.to(device)
        outputs = model(x)
        loss, ratio = model.my_loss(outputs, y, data.group_id)
        print(loss)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

 44%|████▍     | 4/9 [00:01<00:01,  3.13it/s]

tensor(inf, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)


 78%|███████▊  | 7/9 [00:01<00:00,  5.89it/s]

tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)


 44%|████▍     | 4/9 [00:01<00:01,  2.96it/s]

tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 9/9 [00:01<00:00,  4.54it/s]


tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)


  0%|          | 0/9 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [174]:
outputs = torch.arange(0,10).to(device)
y = torch.arange(0,10).to(device)
group_id = torch.ones(10).to(device)
model = TBNet().to(device)
loss = model.my_loss(outputs, y, group_id)

tensor([1, 2, 1, 3, 2, 1, 4, 3, 2, 1, 5, 4, 3, 2, 1, 6, 5, 4, 3, 2, 1, 7, 6, 5,
        4, 3, 2, 1, 8, 7, 6, 5, 4, 3, 2, 1, 9, 8, 7, 6, 5, 4, 3, 2, 1],
       device='cuda:0')
Total valid pairs: 45.000, reversed: 45.000, reverse ratio: 1.000


In [30]:
exp(500)

NameError: name 'exp' is not defined